In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
#!/usr/bin/python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load price data from csv
DATASET=pd.read_csv(r'C:\Users\anaal\DATASET3.csv')

In [20]:
DATASET['settlementdate']=pd.to_datetime(DATASET['settlementdate'],dayfirst=True )
DATASET= DATASET.sort_values(by="settlementdate")

print(DATASET.loc[0,'Dates'],DATASET.loc[DATASET.shape[0]-1,'Dates'])

2019-06-09 2021-03-01


In [21]:
DATASET['Dates'] = pd.to_datetime(DATASET['Dates']).dt.date

In [17]:
pd.date_range(start = '2019-06-09', end = '2021-03-01' ).difference(DATASET.Dates.unique())

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [22]:
DATASET.head()

,Unnamed: 0,settlementdate,runno_x,regionid_x,intervention_x,rrp_x,eep_x,rop_x,apcflag_x,marketsuspendedflag_x,totaldemand_x,demandforecast_x,dispatchablegeneration_x,dispatchableload_x,netinterchange_x,excessgeneration_x,lower5minlocaldispatch_x,lower60seclocaldispatch_x,lower6seclocaldispatch_x,raise5minlocaldispatch_x,raise60seclocaldispatch_x,raise6seclocaldispatch_x,aggregatedispatcherror_x,availablegeneration_x,availableload_x,initialsupply_x,clearedsupply_x,lowerreglocaldispatch_x,raisereglocaldispatch_x,raise6secrrp_x,raise6secrop_x,raise6secapcflag_x,raise60secrrp_x,raise60secrop_x,raise60secapcflag_x,raise5minrrp_x,raise5minrop_x,raise5minapcflag_x,raiseregrrp_x,raiseregrop_x,...,raise60secrop_y,raise60secapcflag_y,raise5minrrp_y,raise5minrop_y,raise5minapcflag_y,raiseregrrp_y,raiseregrop_y,raiseregapcflag_y,lower6secrrp_y,lower6secrop_y,lower6secapcflag_y,lower60secrrp_y,lower60secrop_y,lower60secapcflag_y,lower5minrrp_y,lower5minrop_y,lower5minapcflag_y,lowerregrrp_y,lowerregrop_y,lowerregapcflag_y,raise6secactualavailability_y,raise60secactualavailability_y,raise5minactualavailability_y,raiseregactualavailability_y,lower6secactualavailability_y,lower60secactualavailability_y,lower5minactualavailability_y,lowerregactualavailability_y,Dates,Time,year,month,day_of_week,rrpOneForward,rrpTwoForward,rrpOneBack,rrpTwoBack,rrpOneBackNSW,rrpTwoBackNSW,newTime
2,4,2019-06-09 00:05:00,1,QLD1,0,73.04979,0.0,73.04979,0,0,5756.78,-28.17822,6548.76,0.0,791.97,0.0,0.0,0.0,0.0,121.00,82.1,73.0,-8.84390,9897.150,0.0,5807.87842,5770.24,125.80,71.13,1.45,1.45,0,1.00,1.00,0,0.99,0.99,0,23.7300,23.7300,...,1.00,0,0.99,0.99,0,23.7300,23.7300,0,0.00,0.00,0,0.08,0.08,0,0.19,0.19,0,80.00000,80.00000,0,169.26258,218.73756,462.73756,137.843893,280.0,420.0,230.0,56.670325,2019-06-09,00:05:00,2019,6,6,77.44482,77.73004,25.49994,25.49994,27.07695,27.07695,1.0
3,5,2019-06-09 00:10:00,1,QLD1,0,77.44482,0.0,77.44482,0,0,5762.13,-8.98486,6495.07,0.0,732.94,0.0,0.0,0.0,0.0,123.00,77.0,68.0,0.00000,9899.181,0.0,5784.75830,5773.43,121.80,51.00,1.50,1.50,0,1.00,1.00,0,0.99,0.99,0,20.0000,20.0000,...,1.00,0,0.99,0.99,0,20.0000,20.0000,0,0.00,0.00,0,0.11,0.11,0,0.19,0.19,0,50.12914,50.12914,0,175.00000,267.00000,519.10633,121.259542,446.0,586.0,296.0,83.000024,2019-06-09,00:10:00,2019,6,6,77.73004,77.39365,73.04979,25.49994,82.63841,27.07695,2.0
4,6,2019-06-09 00:15:00,1,QLD1,0,77.73004,0.0,77.73004,0,0,5725.16,-9.62549,6457.46,0.0,732.30,0.0,0.0,0.0,0.0,121.38,77.0,68.0,-22.39729,9895.838,0.0,5769.14795,5736.46,116.40,51.00,1.50,1.50,0,1.00,1.00,0,0.99,0.99,0,20.0001,20.0001,...,1.00,0,0.99,0.99,0,20.0001,20.0001,0,0.03,0.03,0,0.05,0.05,0,0.19,0.19,0,23.73000,23.73000,0,175.00000,267.00000,535.00000,145.079149,446.0,586.0,296.0,82.843774,2019-06-09,00:15:00,2019,6,6,77.39365,76.70402,77.44482,73.04979,87.00000,82.63841,3.0
5,7,2019-06-09 00:20:00,1,QLD1,0,77.39365,0.0,77.39365,0,0,5688.49,-11.57324,6431.56,0.0,743.07,0.0,0.0,0.0,0.0,123.00,77.0,68.0,-20.57330,9891.610,0.0,5731.33936,5700.27,126.57,51.00,1.50,1.50,0,1.00,1.00,0,0.99,0.99,0,20.0000,20.0000,...,1.00,0,0.99,0.99,0,20.0000,20.0000,0,0.03,0.03,0,0.05,0.05,0,0.19,0.19,0,23.73000,23.73000,0,175.00000,267.00000,535.00000,145.773460,446.0,586.0,296.0,64.863695,2019-06-09,00:20:00,2019,6,6,76.70402,71.74000,77.73004,77.44482,87.00000,87.00000,4.0
6,8,2019-06-09 00:25:00,1,QLD1,0,76.70402,0.0,76.70402,0,0,5640.21,-20.36035,6428.85,0.0,788.64,0.0,0.0,0.0,0.0,123.00,77.0,68.0,-18.22192,9891.691,0.0,5690.65430,5653.72,139.80,51.00,1.50,1.50,0,1.49,1.49,0,0.99,0.99,0,20.0000,20.0000,...,1.49,0,0.99,0.99,0,20.0000,20.0000,0,0.00,0.00,0,0.05,0.05,0,0.19,0.19,0,32.89803,32.89803,0,175.00000,267.00000,535.00000,146.428094,446.0,586.0,296.0,76.382947,2019-06-09,00:25:00,2019,6,6,71.74000,75.75269,77.39365,77.73004,87.00000,87.00000,5.0


In [22]:
DATASET['quarters']=pd.cut(DATASET['month'],bins=[1,4,8, 10,12],include_lowest=True,right=False, labels=['first','second','third','fourth'])
DATASET['quarters']=pd.cut(DATASET['month'],bins=[1,4,8, 10,12],include_lowest=True,right=False, labels=['first','second','third','fourth'])
DATASET['weekclass']=pd.cut(DATASET['day_of_week'],bins=[-1,0,4, 5,6],include_lowest=False,right=True, labels=['Monday','Midweek','Saturday','Sunday'])
DATASET['classesAUD']=pd.cut(DATASET['rrpOneForward'],bins=[-1000,0,32,60,100,15001],include_lowest=True,right=False, labels=['negative','L','normal','high','spike'])


DATASET['timeClass']=pd.cut(DATASET['newTime'],bins=[0,50,120,180,250,288],include_lowest=True,right=False, labels=['low','M1','high','M2','low2'])
#print(merged_data['timeClass'])

DATASET['RMQ']=DATASET.clearedsupply_x/DATASET.availablegeneration_x

In [23]:
DATASET['RMNSW']=DATASET.clearedsupply_y/DATASET.availablegeneration_y

In [24]:
DATASET['RMNSW'].isna()

0         False
1         False
2         False
3         False
4         False
          ...  
180863    False
180864    False
180865    False
180866    False
180867    False
Name: RMNSW, Length: 180868, dtype: bool

In [25]:
data_encoded1=pd.get_dummies(DATASET['quarters'],prefix='quarters')
data_encoded2=pd.get_dummies(DATASET['weekclass'],prefix='weekclass')
data_encoded3=pd.get_dummies(DATASET['classesAUD'],prefix='classesAUD')
data_encoded4=pd.get_dummies(DATASET['timeClass'],prefix='timeClass')

datafinal=pd.concat([DATASET,data_encoded1,data_encoded2,data_encoded3,data_encoded4],axis=1)
datafinal['timeindex']=(datafinal['newTime'] +1)/288

In [26]:
print(datafinal.columns)

y= datafinal.loc[:,datafinal.columns.isin(['classesAUD_negative','classesAUD_L', 'classesAUD_normal', 'classesAUD_high','classesAUD_spike'])]

#x=datafinal.loc[:,datafinal.columns.isin(['rrp_x', 'totaldemand_x', 'clearedsupply_x', 'rrp_y', 'rrpOneBack',
      # 'RMQ', 'RMNSW','totaldemand_y'])]
x=datafinal.loc[:,datafinal.columns.isin(['rrp_x','RMQ','rrp_y','timeindex'])]

Index(['Unnamed: 0', 'settlementdate', 'runno_x', 'regionid_x',
       'intervention_x', 'rrp_x', 'eep_x', 'rop_x', 'apcflag_x',
       'marketsuspendedflag_x',
       ...
       'classesAUD_L', 'classesAUD_normal', 'classesAUD_high',
       'classesAUD_spike', 'timeClass_low', 'timeClass_M1', 'timeClass_high',
       'timeClass_M2', 'timeClass_low2', 'timeindex'],
      dtype='object', length=157)


In [151]:
print(datafinal.columns)

Index(['Unnamed: 0', 'settlementdate', 'runno_x', 'regionid_x',
       'intervention_x', 'rrp_x', 'eep_x', 'rop_x', 'apcflag_x',
       'marketsuspendedflag_x',
       ...
       'classesAUD_negative', 'classesAUD_L', 'classesAUD_normal',
       'classesAUD_high', 'classesAUD_spike', 'timeClass_low', 'timeClass_M1',
       'timeClass_high', 'timeClass_M2', 'timeClass_low2'],
      dtype='object', length=158)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test= train_test_split(x, y, shuffle=False,test_size=0.5,random_state = 42)

print(Y_train.shape)
print

print(X_train.head())

(114392, 5)
      rrp_x     rrp_y       RMQ  timeindex
2  73.04979  82.63841  0.583020   0.006944
3  77.44482  87.00000  0.583223   0.010417
4  77.73004  87.00000  0.579684   0.013889
5  77.39365  87.00000  0.576273   0.017361
6  76.70402  86.84650  0.571563   0.020833


In [203]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2

test = SelectKBest(score_func=mutual_info_classif, k=20)
print(x.columns)
fit = test.fit(x, datafinal['classesAUD'])
# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)
f = fit.get_support() 
print(f)
bestFeatures=x.columns[f]
print(bestFeatures)


Index(['runno_x', 'intervention_x', 'rrp_x', 'eep_x', 'apcflag_x',
       'marketsuspendedflag_x', 'totaldemand_x', 'demandforecast_x',
       'dispatchablegeneration_x', 'dispatchableload_x',
       ...
       'quarters_fourth', 'weekclass_Monday', 'weekclass_Midweek',
       'weekclass_Saturday', 'weekclass_Sunday', 'timeClass_low',
       'timeClass_M1', 'timeClass_high', 'timeClass_M2', 'timeClass_low2'],
      dtype='object', length=133)
[1.917e-02 4.769e-04 8.702e-01 6.114e-04 1.401e-03 3.714e-04 3.812e-01
 2.591e-01 3.584e-01 5.690e-02 2.678e-01 0.000e+00 2.365e-02 1.443e-01
 1.189e-01 1.354e-01 1.676e-01 1.546e-01 2.154e-01 2.886e-01 5.630e-02
 3.871e-01 3.704e-01 1.183e-01 1.206e-01 1.933e-01 1.954e-01 1.340e-03
 1.836e-01 1.816e-01 8.414e-04 8.808e-02 8.648e-02 9.955e-04 3.417e-01
 3.418e-01 0.000e+00 2.318e-01 2.328e-01 0.000e+00 2.494e-01 2.494e-01
 0.000e+00 1.637e-01 1.652e-01 9.361e-04 2.168e-01 2.175e-01 8.131e-04
 3.061e-01 3.782e-01 2.678e-01 3.331e-01 3.014e-01 3.081

In [ ]:
from sklearn.preprocessing import RobustScaler
#scale=MinMaxScaler(feature_range=(0,1))
scale=RobustScaler(with_centering=False)

In [27]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf =RandomForestClassifier(n_estimators = 70, random_state = 42,verbose=7,min_samples_leaf=85,max_depth=20)
# Train the model on training data





rf.fit(X_train, Y_train);
y_pred = rf.predict(X_test)
# report precision score
from sklearn.metrics import precision_score
print('precision', precision_score(Y_test, y_pred,average=None))
# recall score
from sklearn.metrics import recall_score
print('recall', recall_score(Y_test, y_pred,average=None))
# f1 score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
Y_testNP=Y_test.to_numpy()
print(confusion_matrix(Y_testNP.argmax(axis=1),y_pred.argmax(axis=1)))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 70
building tree 2 of 70


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s


building tree 3 of 70
building tree 4 of 70


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.9s remaining:    0.0s


building tree 7 of 70
building tree 8 of 70
building tree 9 of 70
building tree 10 of 70
building tree 11 of 70
building tree 12 of 70
building tree 13 of 70
building tree 14 of 70
building tree 15 of 70
building tree 16 of 70
building tree 17 of 70
building tree 18 of 70
building tree 19 of 70
building tree 20 of 70
building tree 21 of 70
building tree 22 of 70
building tree 23 of 70
building tree 24 of 70
building tree 25 of 70
building tree 26 of 70
building tree 27 of 70
building tree 28 of 70
building tree 29 of 70
building tree 30 of 70
building tree 31 of 70
building tree 32 of 70
building tree 33 of 70
building tree 34 of 70
building tree 35 of 70
building tree 36 of 70
building tree 37 of 70
building tree 38 of 70
building tree 39 of 70
building tree 40 of 70
building tree 41 of 70
building tree 42 of 70
building tree 43 of 70
building tree 44 of 70
building tree 45 of 70
building tree 46 of 70
building tree 47 of 70
building tree 48 of 70
building tree 49 of 70
building tree 

[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:   12.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s


precision [0.69008516 0.89969181 0.82378702 0.67936563 0.75736568]
recall [0.64625463 0.60609767 0.95677401 0.7307621  0.53227771]
[[ 2385  1065    50     7     4]
 [ 1504 21311 12306    25    15]
 [  221  1283 64145  1185   209]
 [   46    14  1060  3941   332]
 [  574    14   305   643  1748]]


[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:    1.8s finished


In [212]:
#%% RANDOM FOREST

#This tuning only ended up with 74%  Accuracy and was left of the final 5
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.metrics import confusion_matrix, zero_one_loss,accuracy_score,accuracy_score,f1_score,precision_score,recall_score
accuracy_av=metrics.make_scorer(metrics.accuracy_score)
re_av=metrics.make_scorer(metrics.recall_score,average='weighted')
n_estimators=[70,80,90]
max_depth=[10,15,20]
min_samples_leaf=[75,80,85]
parametersGrid = { "n_estimators": n_estimators,"max_depth":max_depth,"min_samples_leaf":min_samples_leaf}
rfclf = RandomForestClassifier(random_state=42)
gridrf = GridSearchCV(estimator=rfclf, param_grid=parametersGrid, scoring=re_av, cv=10,verbose=100)
gridrf=gridrf.fit(X_train, Y_train)


print(gridrf.best_params_ )


Fitting 10 folds for each of 27 candidates, totalling 270 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] max_depth=10, min_samples_leaf=75, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=75, n_estimators=70, score=0.870, total=   9.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.3s remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=75, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=75, n_estimators=70, score=0.849, total=   9.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.6s remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=75, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=75, n_estimators=70, score=0.893, total=   9.7s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   28.4s remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=75, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=75, n_estimators=70, score=0.944, total=

[CV]  max_depth=10, min_samples_leaf=80, n_estimators=70, score=0.902, total=   9.4s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  6.2min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=80, n_estimators=70, score=0.836, total=   9.7s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  6.4min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=80, n_estimators=70, score=0.863, total=   9.5s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:  6.6min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=80, n_estimators=70, score=0.899, total=   9.6s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  6.7min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=10, min_samples_leaf=80, n_estimators=70

[CV]  max_depth=10, min_samples_leaf=85, n_estimators=70, score=0.876, total=   9.5s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed: 12.5min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=10, min_samples_leaf=85, n_estimators=80, score=0.870, total=  10.7s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed: 12.6min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=10, min_samples_leaf=85, n_estimators=80, score=0.847, total=  10.6s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 12.8min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=10, min_samples_leaf=85, n_estimators=80, score=0.894, total=  11.0s
[Parallel(n_jobs=1)]: Done  73 out of  73 | elapsed: 13.0min remaining:    0.0s
[CV] max_depth=10, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=10, min_samples_leaf=85, n_estimators=80

[CV]  max_depth=15, min_samples_leaf=75, n_estimators=80, score=0.899, total=  12.5s
[CV] max_depth=15, min_samples_leaf=75, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=75, n_estimators=80, score=0.888, total=  12.6s
[CV] max_depth=15, min_samples_leaf=75, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=75, n_estimators=80, score=0.875, total=  12.4s
[CV] max_depth=15, min_samples_leaf=75, n_estimators=90 ..............
[CV]  max_depth=15, min_samples_leaf=75, n_estimators=90, score=0.870, total=  14.5s
[CV] max_depth=15, min_samples_leaf=75, n_estimators=90 ..............
[CV]  max_depth=15, min_samples_leaf=75, n_estimators=90, score=0.848, total=  14.5s
[CV] max_depth=15, min_samples_leaf=75, n_estimators=90 ..............
[CV]  max_depth=15, min_samples_leaf=75, n_estimators=90, score=0.894, total=  14.3s
[CV] max_depth=15, min_samples_leaf=75, n_estimators=90 ..............
[CV]  max_depth=15, min_samples_leaf=75, n_estimators=90, score=

[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=0.870, total=  13.7s
[CV] max_depth=15, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=0.847, total=  13.6s
[CV] max_depth=15, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=0.893, total=  14.6s
[CV] max_depth=15, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=0.944, total=  14.7s
[CV] max_depth=15, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=0.903, total=  13.1s
[CV] max_depth=15, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=0.837, total=  13.3s
[CV] max_depth=15, min_samples_leaf=85, n_estimators=80 ..............
[CV]  max_depth=15, min_samples_leaf=85, n_estimators=80, score=

[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=0.943, total=  11.9s
[CV] max_depth=20, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=0.902, total=  11.1s
[CV] max_depth=20, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=0.837, total=  11.1s
[CV] max_depth=20, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=0.864, total=  11.0s
[CV] max_depth=20, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=0.898, total=  10.8s
[CV] max_depth=20, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=0.887, total=  11.1s
[CV] max_depth=20, min_samples_leaf=80, n_estimators=70 ..............
[CV]  max_depth=20, min_samples_leaf=80, n_estimators=70, score=

[CV]  max_depth=20, min_samples_leaf=85, n_estimators=90, score=0.863, total=  14.3s
[CV] max_depth=20, min_samples_leaf=85, n_estimators=90 ..............
[CV]  max_depth=20, min_samples_leaf=85, n_estimators=90, score=0.898, total=  14.1s
[CV] max_depth=20, min_samples_leaf=85, n_estimators=90 ..............
[CV]  max_depth=20, min_samples_leaf=85, n_estimators=90, score=0.886, total=  14.3s
[CV] max_depth=20, min_samples_leaf=85, n_estimators=90 ..............
[CV]  max_depth=20, min_samples_leaf=85, n_estimators=90, score=0.876, total=  14.2s
[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed: 54.7min finished
{'max_depth': 20, 'min_samples_leaf': 75, 'n_estimators': 80}


In [15]:
print(gridrf.best_params_ )

{'max_depth': 20, 'min_samples_leaf': 85, 'n_estimators': 70}


In [12]:
print(datafinal.shape)
listHolidays=[np.datetime64('2020-12-28'),np.datetime64('2019-12-25'),np.datetime64('2019-12-26'),np.datetime64('2019-12-31'),np.datetime64('2020-12-25'),np.datetime64('2020-01-01'),np.datetime64('2020-01-27'),np.datetime64('2020-01-02'),np.datetime64('2020-01-03'),np.datetime64('2020-04-10'),np.datetime64('2020-04-13'),np.datetime64('2021-01-26'),np.datetime64('2021-01-01')]
datafinal['Dates']=pd.to_datetime(datafinal['Dates'])
datafinal=datafinal[~datafinal.Dates.isin(listHolidays)]
print(datafinal.shape)

(228784, 157)
(223309, 157)


In [102]:
correlation=datafinal.corr('pearson')
correlation.to_csv(r'C:\Users\anaal\correlation.csv')

In [28]:
datafinal.to_csv(r'C:\Users\anaal\datafinal.csv')